In [1]:
import sys
sys.path.append("/Users/jrand/git-repos/AIE7/11_Certification_Challenge/backend/src")

In [15]:
import pandas as pd

experiment_id = "096bef9f-dff4-4a70-9cb2-0735ac3ad1e7"

score_df = pd.read_csv(f"../data/experiments/score_df_{experiment_id}.csv").drop(columns=["Unnamed: 0"])

score_df.head()

,context_recall,faithfulness,factual_correctness,answer_relevancy,experiment_name
0,1.0,0.916667,0.83,0.985142,naive_chunker_cosine
1,1.0,0.833333,0.40,0.830221,naive_chunker_cosine
2,1.0,1.000000,0.89,0.872344,naive_chunker_cosine
3,1.0,1.000000,0.90,0.944921,naive_chunker_cosine
4,1.0,0.769231,0.67,0.979184,naive_chunker_cosine


In [11]:
score_aggs = score_df.groupby('experiment_name').mean()
score_aggs

,context_recall,faithfulness,factual_correctness,answer_relevancy
experiment_name,,,,
markdown_chunker_cosine,0.596753,0.792318,0.578182,0.853453
markdown_chunker_multi_query,0.691883,0.855392,0.620000,0.941706
naive_chunker_cosine,0.950758,0.846997,0.723810,0.812433
naive_chunker_multi_query,0.960606,0.832359,0.699524,0.817645


In [19]:
from collections import defaultdict
from langsmith import Client
import os
from tqdm import tqdm
from mimi.evals.ragas_eval import generate_all_invocables

client = Client()

invocables = generate_all_invocables()
tags = list(invocables.keys())
by_tag = defaultdict(dict)

for t in tqdm(tags):
    filter = f"and(has(tags, '{t}'), has(tags, '{experiment_id}'))"
    print(filter)
    by_tag[t] = client.get_run_stats(
        project_names=[os.environ["LANGSMITH_PROJECT"]],
        # filter syntax lets you match an element in the tags array
        filter=filter
    )

  0%|          | 0/4 [00:00<?, ?it/s]

and(has(tags, 'naive_chunker_cosine'), has(tags, '096bef9f-dff4-4a70-9cb2-0735ac3ad1e7'))


 25%|██▌       | 1/4 [00:02<00:07,  2.58s/it]

and(has(tags, 'naive_chunker_multi_query'), has(tags, '096bef9f-dff4-4a70-9cb2-0735ac3ad1e7'))


 50%|█████     | 2/4 [00:03<00:03,  1.85s/it]

and(has(tags, 'markdown_chunker_cosine'), has(tags, '096bef9f-dff4-4a70-9cb2-0735ac3ad1e7'))


 75%|███████▌  | 3/4 [00:05<00:01,  1.66s/it]

and(has(tags, 'markdown_chunker_multi_query'), has(tags, '096bef9f-dff4-4a70-9cb2-0735ac3ad1e7'))


100%|██████████| 4/4 [00:06<00:00,  1.75s/it]


In [20]:
langsmith_df = pd.DataFrame(by_tag).T
langsmith_df

,run_count,latency_p50,latency_p99,first_token_p50,first_token_p99,total_tokens,prompt_tokens,completion_tokens,median_tokens,completion_tokens_p50,...,last_run_start_time,feedback_stats,run_facets,error_rate,streaming_rate,total_cost,prompt_cost,completion_cost,cost_p50,cost_p99
naive_chunker_cosine,176,1.021,17.08425,None,None,60615,55863,4752,None,None,...,2025-08-03T21:39:30.842980,None,"[{'key': 'name', 'value': 'StrOutputParser', '...",0.0,0.0,0.023574,0.015971,0.007603,0.001122,0.0017
naive_chunker_multi_query,330,0.8105,21.21216,None,None,68216,61538,6678,None,None,...,2025-08-03T21:40:31.916139,None,"[{'key': 'name', 'value': 'VectorStoreRetrieve...",0.0,0.0,0.024112,0.016201,0.00791,0.000084,0.001742
markdown_chunker_cosine,176,0.7075,5.57225,None,None,17306,14416,2890,None,None,...,2025-08-03T21:38:42.701835,None,"[{'key': 'name', 'value': 'StrOutputParser', '...",0.0,0.0,0.01039,0.005766,0.004624,0.000495,0.000738
markdown_chunker_multi_query,330,0.9535,11.70478,None,None,29340,23237,6103,None,None,...,2025-08-03T21:40:46.033890,None,"[{'key': 'name', 'value': 'VectorStoreRetrieve...",0.0,0.0,0.01549,0.008523,0.006968,0.000082,0.000896


In [21]:
score_aggs_combined = pd.concat([score_aggs, -langsmith_df[['latency_p50', 'latency_p99', 'cost_p50', 'cost_p99']]], axis=1)
score_aggs_combined

,context_recall,faithfulness,factual_correctness,answer_relevancy,latency_p50,latency_p99,cost_p50,cost_p99
markdown_chunker_cosine,0.596753,0.792318,0.578182,0.853453,-0.7075,-5.57225,-0.000495,-0.000738
markdown_chunker_multi_query,0.691883,0.855392,0.620000,0.941706,-0.9535,-11.70478,-0.000082,-0.000896
naive_chunker_cosine,0.950758,0.846997,0.723810,0.812433,-1.021,-17.08425,-0.001122,-0.0017
naive_chunker_multi_query,0.960606,0.832359,0.699524,0.817645,-0.8105,-21.21216,-0.000084,-0.001742


In [22]:
score_aggs_combined.rank(ascending=False)

,context_recall,faithfulness,factual_correctness,answer_relevancy,latency_p50,latency_p99,cost_p50,cost_p99
markdown_chunker_cosine,4.0,4.0,4.0,2.0,1.0,1.0,3.0,1.0
markdown_chunker_multi_query,3.0,1.0,3.0,1.0,3.0,2.0,1.0,2.0
naive_chunker_cosine,2.0,2.0,1.0,4.0,4.0,3.0,4.0,3.0
naive_chunker_multi_query,1.0,3.0,2.0,3.0,2.0,4.0,2.0,4.0


In [23]:
# aggregated rankings for evals only
score_aggs.rank(ascending=False).mean(axis=1).sort_values(ascending=True)

experiment_name
markdown_chunker_multi_query    2.00
naive_chunker_cosine            2.25
naive_chunker_multi_query       2.25
markdown_chunker_cosine         3.50
dtype: float64

In [24]:
# aggregated rankings for cost/latency only
(-langsmith_df[['latency_p50', 'latency_p99', 'cost_p50', 'cost_p99']]).rank(ascending=False).mean(axis=1).sort_values(ascending=True)

markdown_chunker_cosine         1.5
markdown_chunker_multi_query    2.0
naive_chunker_multi_query       3.0
naive_chunker_cosine            3.5
dtype: float64

In [25]:
# aggregated rankings for both evals and cost/latency
score_aggs_combined.rank(ascending=False).mean(axis=1).sort_values(ascending=True)

markdown_chunker_multi_query    2.000
markdown_chunker_cosine         2.500
naive_chunker_multi_query       2.625
naive_chunker_cosine            2.875
dtype: float64

# Conclusion

The winner of my experiments is `multi_query`, which provides the optimal balance of quality, latency, and cost. The cell below provides a complete picture of how each retriever ranks compared to all other retrievers on 8 different dimensions.

The rankings largely meet expectations. For example, bm25 is fast but has the worst accuracy. The ensemble is slow and expensive, but did not produce enough accuracy boost to justify.

In [26]:
score_aggs_combined.rank(ascending=False)

,context_recall,faithfulness,factual_correctness,answer_relevancy,latency_p50,latency_p99,cost_p50,cost_p99
markdown_chunker_cosine,4.0,4.0,4.0,2.0,1.0,1.0,3.0,1.0
markdown_chunker_multi_query,3.0,1.0,3.0,1.0,3.0,2.0,1.0,2.0
naive_chunker_cosine,2.0,2.0,1.0,4.0,4.0,3.0,4.0,3.0
naive_chunker_multi_query,1.0,3.0,2.0,3.0,2.0,4.0,2.0,4.0
